<a href="https://colab.research.google.com/github/bosunKwak/BigData/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Overall Process
<img width="380" alt="스크린샷 2022-04-19 오후 11 29 44" src="https://user-images.githubusercontent.com/87002218/164027812-736c6c74-e20b-4fcc-a057-dc2c1524d2c2.png">


"here is the example"
1) Tokenize : here, is , the ,example
- 문장을 토큰 시퀀스로 분석하는 과정
- 토큰의 분리기준은 다를 수 있지만 문장을 구성하는 기본단위 -ex) 단어

2) Vocabulary : 475, 21, 2, 30, 5297
- 모은 단어의 토큰들을 합쳐 하나의 list를 만듦(어휘 집합 == 토크나이즈 수행 후 중복을 제거한 토큰들의 집합)
- 각 단어를 문자 단위 ID 시퀀스로 변환하기 위해 사용

3) Embedding : [[0.81,0.27,0.59], [ , , ], [ , , ], [ , , ]]
- 컴퓨터는 인간이 사용하는 자연어를 있는 그대로 이해하는 것이 아니라 숫자를 계산
- 사람이 쓰는 자연어를 기계가 이해할 수 잇는 숫자의 나열인 벡터로 바꾼 결과/ 일련의 과정 전체
- 단어나 문장을 각각 벡터로 변환해 벡터 공간(vector space)로 끼워넣는다(embed) 

# Text classification
- [pytorch를 이용해서 모델을 개발하는 전반적인 과정](https://github.com/bosunKwak/BigData/blob/main/overall_process.ipynb)을 text data를 가지고 실습
- [tutorial 참고](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html)
- data preparation & model development 
- [dataset](https://pytorch.org/text/stable/datasets.html#ag-news)

## 1. Data Preparation

### - Access to the raw dataset iterators

In [1]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = iter(AG_NEWS(split='train'))
# iter : 순환 가능한 객체로 만드는 것

29.5MB [00:00, 41.1MB/s]


In [10]:
next(train_iter)
# 하나씩 꺼내옴

(3,
 'Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insurance benefits available through his employer. But, at 32, "buying insurance was the furthest thing from my mind," says Riley.')

In [11]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# tokenizer 선정 
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

# tokenizer 수행하는 함수
def yield_tokens(data_iter):
  # (category , text) : 뒤에 있는 text만 가져옴 
    for _, text in data_iter:
        yield tokenizer(text) # text를 tokenizer로 token화

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"]) # tokenize된 token들이 vocab에 들어오고, 새로운 단어의 경우 <unknown>처리
vocab.set_default_index(vocab["<unk>"]) # set_default_index : unknown 이라는 token을 index로 지정하겠다는 의미

In [19]:
vocab.lookup_token(4)

'to'

In [22]:
vocab(['here','is','an','example','to'])

[475, 21, 30, 5297, 4]

### - Prepare data processing pipelines
text가 들어왔을 때, 이것을 어떻게 가공할 것인지에 대한 절차 정의

In [23]:
# text를 받았을 때 처리하는 pipeline
text_pipeline = lambda x: vocab(tokenizer(x)) # x를 받아 tokenizer를 통해 토큰으로 바꾸고 다시 vocab으로

# label 정보를 받았을 때 처리하는 pipeline
label_pipeline = lambda x: int(x) - 1 # 1~4 -> 0~3

In [26]:
text_pipeline('here is an example')

[475, 21, 30, 5297]

### - Generate data batch and iterator


collate_fn 
- batch를 통해 collate_fn로 전달되면 데이터를 처리
- collate : 종이를 순서에 맞춰서 정렬을 하는 것 


In [27]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# batch : Dataloader에서 받아온 batch 
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0] #offsets : 문장이 시작되는 지점 (첫번째 문장의 offset : 0 )
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)